In [1]:
import pandas as pd
import haversine as hs

In [2]:
# Load Data
CSV_FILE_PATH = './data/sample_taxi_after_clean.csv'
df = pd.read_csv(CSV_FILE_PATH)
df.head()

,taxi_id,time,lon,lat,is_passenger,speed
0,22224,00:00:12,114.039169,22.565283,0,45
1,22224,00:00:27,114.040001,22.563417,0,69
2,22224,00:00:42,114.041298,22.561033,0,70
3,22224,00:00:57,114.042366,22.559084,0,69
4,22224,00:01:23,114.043114,22.556850,0,23


In [3]:
# Type Conversion
df.lon = df.lon.astype('double')
df.lat = df.lat.astype('double')

In [4]:
# Statistical Data
taxi_counts = dict(df.taxi_id.value_counts())

In [5]:
id_list = list(taxi_counts.keys())

In [8]:
# Obtain OD latitude and longitude, and calculate OD polyline distance
OD_list = []
OD_path = []
for id in id_list:
    id_df = df[df.taxi_id == id]
    id_df = id_df.sort_values(by='time')
#     id_df = id_df[id_df.duplicated('time')== False]
    passenger_check = False
    distance = 0
    last_point = [0, 0, 0]
#     print(id_df.shape)
    for index, row in id_df.iterrows():
        if (len(row)==0):
            print(row)
            continue
        if (passenger_check and row['is_passenger'] == 0):
            passenger_check = False
            loc1 = (last_point[0], last_point[1])
            if (last_point == [0, 0, 0]):
                loc1 = (OD_path[0][1], OD_path[0][2])
            loc2 = (row['lon'], row['lat'])
            distance += hs.haversine(loc1, loc2)
            if (distance > 100):
                print(distance)
            
            OD_path.append([row['time'], row['lon'], row['lat']])
            OD_path.append(distance)
            OD_list.append(OD_path.copy())
            OD_path = []
            distance = 0
            last_point = [0, 0, 0]
        elif ((not passenger_check) and row['is_passenger'] == 1):
            passenger_check = True
            OD_path = []
            OD_path.append([row['time'], row['lon'], row['lat']])
#             print(OD_path)
            last_point = [0, 0, 0]
        elif passenger_check and row['is_passenger'] == 1 and last_point != [0, 0, 0]:
            loc1 = (last_point[0], last_point[1])
            loc2 = (row['lon'], row['lat'])
            distance += hs.haversine(loc1, loc2)
            if (distance > 100):
                print(distance)
            last_point[0] = row['lon']
            last_point[1] = row['lat']
            last_point[2] = row['time']

In [10]:
# Format conversion
r_df1 = pd.DataFrame(
    columns=['from_time', 'lng1', 'lat1', 'to_time', 'lng2', 'lat2', 'distance'])
for i in OD_list:
    slide_series = pd.Series(
        {'from_time': i[0][0],
         'lng1': i[0][1],
         'lat1': i[0][2],
         'to_time': i[1][0],
         'lng2': i[1][1],
         'lat2': i[1][2],
         'distance': i[2]})
    r_df1 = r_df1.append(slide_series, ignore_index=True)



In [ ]:
r_df1.describe()

In [ ]:
# save data
r_df1.to_csv("./data/OD_distance_data.csv", index=False, sep=',')